In [1]:
import numpy as np
import scipy as sp
from tqdm import tqdm
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
import random
import sys
sys.path.append('../src')
from models import get_matrices_from_term_lists, \
    filter_terms_not_in_wemodel

In [2]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')

'''
# Caliskan GloVe
glove_file = '../data/external/glove.6B/glove.6B.50d.txt'
_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')
we_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')
print('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')
'''

loading done!
Total words: 312425


"\n# Caliskan GloVe\nglove_file = '../data/external/glove.6B/glove.6B.50d.txt'\n_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')\nwe_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')\nprint('loading done!')\nprint(f'Total words: {len(we_model.wv.vocab)}')\n"

In [4]:
X_terms = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 
           'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil','lilac',
           'pansy','tulip','buttercup','daisy','lily','penny','violet','carnation', 'gladiola',
           'magnolia','petunia','zinnia']
Y_terms = ['ant','caterpillar','flea','locust','spider','bedbug','centipede','fly',
          'maggot','tarantula','bee','cockroach','gnat','mosquito','termite','beetle',
          'cricket','hornet','moth','wasp','blackfly','dragonfly','horsefly','roach',
          'weevil']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
X_terms, Y_terms = filter_terms_not_in_wemodel(we_model, X_terms, Y_terms)
A_terms, B_terms = filter_terms_not_in_wemodel(we_model, A_terms, B_terms)

The following terms were removed from the list first_list because they were not found in the we_model: ['gladiola']
The following terms were removed from the list second_list because they were not found in the we_model: []
The following terms were removed from the second list to balance the length of the lists: ['ant']
The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []


In [9]:
# Fastest version, 10000 words -> 1 minute
# (Possible TODO) May be able to add minimal speedup with itemgetter 
# (see https://stackoverflow.com/questions/18453566/python-dictionary-get-list-of-values-for-list-of-keys)
# to speed up creation of word matrices in get_matrices_from_term_lists
def get_test_stat(wv_obj, X_terms, Y_terms, A_terms, B_terms):  
    [X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, X_terms, Y_terms, A_terms, B_terms)
    cosine_sim_XA = cosine_similarity(X_mtx, A_mtx)
    cosine_sim_XB = cosine_similarity(X_mtx, B_mtx)
    mean_over_Xa = np.mean(cosine_sim_XA, axis=1)
    mean_over_Xb = np.mean(cosine_sim_XB, axis=1)
    s_for_X_words = mean_over_Xa - mean_over_Xb
    # shape is (24,) or (|X_terms|,)

    cosine_sim_YA = cosine_similarity(Y_mtx, A_mtx)
    cosine_sim_YB = cosine_similarity(Y_mtx, B_mtx)
    mean_over_Ya = np.mean(cosine_sim_YA, axis=1)
    mean_over_Yb = np.mean(cosine_sim_YB, axis=1)
    s_for_Y_words = mean_over_Ya - mean_over_Yb
    test_stat = np.mean(s_for_X_words) - np.mean(s_for_Y_words)
    return test_stat

'\nvocab_list = list(we_model.wv.vocab)\ntest_stats = []\nfor i in tqdm(range(10000)):\n    X_sample = random.sample(vocab_list, len(X_terms))\n    Y_sample = random.sample(vocab_list, len(Y_terms))\n    A_sample = random.sample(vocab_list, len(A_terms))\n    B_sample = random.sample(vocab_list, len(B_terms))\n    test_stat = get_test_stat(we_model, X_sample, Y_sample, A_sample, B_sample)\n    test_stats.append(test_stat)'

In [33]:
# This cell works too. It takes twice as long as the cell above,
# but if we want to try to vectorize the outer loop, then 
# we will probably have to use this version
def calculate_association_metric_for_target_word(word_vec, A_mtx, B_mtx):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    A_dot_v = np.dot(A_mtx, word_vec)
    B_dot_v = np.dot(B_mtx, word_vec)
    A_norms = np.multiply(np.linalg.norm(A_mtx, axis=1), np.linalg.norm(word_vec))
    B_norms = np.multiply(np.linalg.norm(B_mtx, axis=1), np.linalg.norm(word_vec))
    A_cosines = np.divide(A_dot_v, A_norms)
    B_cosines = np.divide(B_dot_v, B_norms)
    return np.mean(A_cosines) - np.mean(B_cosines)

for i in tqdm(range(10000)):
    [X_mtx, Y_mtx, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, X_terms, Y_terms, A_terms, B_terms)
    X_associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    Y_associations = np.apply_along_axis(lambda y_vec: calculate_association_metric_for_target_word(y_vec, A_mtx, B_mtx), 1, Y_mtx)
    m = np.mean(X_associations) - np.mean(Y_associations)


 12%|████████▊                                                                   | 1166/10000 [00:12<01:15, 116.98it/s]

KeyboardInterrupt: 

 12%|████████▊                                                                   | 1166/10000 [00:30<01:15, 116.98it/s]

In [1]:
def get_complements(x_union_y):
    '''Generator function that yields pairs of equal-size disjoint subsets
    of x_union_y.
    x_union_y should a set type.'''
    for seq in combinations(x_union_y, len(x_union_y)//2):
        complement = frozenset(x_union_y.difference(seq))
        yield (seq, complement)

def produce_2ndorder_p_value(wv_obj, X_terms, Y_terms, A_terms, B_terms):
    '''Generates the p-value for a set of terms with the word-vector object.
    High-level function; this function should be directly imported into 
    notebooks for experimentation.'''
    x_union_y = set(X_terms).union(set(Y_terms))
    total_terms = len(x_union_y)
    comparison_statistic = produce_test_statistic(wv_obj, X_terms, Y_terms, A_terms, B_terms)
    dist = np.array([])
    for (X_i_terms, Y_i_terms) in tqdm(get_complements(x_union_y), total=num_combinations(total_terms, total_terms/2)):
        test_statistic = produce_test_statistic(wv_obj, X_i_terms, Y_i_terms, A_terms, B_terms)
        dist = np.append(dist, test_statistic)
    return 1 - sp.stats.norm.cdf(comparison_statistic, loc=np.mean(dist), scale=np.std(dist, ddof=1))

In [10]:
def get_n_test_stats(wv_obj, X_terms, Y_terms, A_terms, B_terms, n_samples=1000):
    test_stats = []
    n_targets = len(X_terms)
    n_attributes = len(A_terms)
    assert len(X_terms) == len(Y_terms)
    assert len(A_terms) == len(B_terms)
    vocab_list = list(wv_obj.wv.vocab)
    for i in tqdm(range(n_samples)):
        X_sample = random.sample(vocab_list, k=n_targets)
        Y_sample = random.sample(vocab_list, k=n_targets)
        A_sample = random.sample(vocab_list, k=n_attributes)
        B_sample = random.sample(vocab_list, k=n_attributes)
        test_stat = get_test_stat(we_model, X_sample, Y_sample, A_sample, B_sample)
        test_stats.append(test_stat)
    return test_stats
get_n_sigtests(we_model, X_terms, Y_terms, A_terms, B_terms)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 479.84it/s]


[0.0022969837,
 -0.008106958,
 -0.0020026485,
 -0.003036597,
 -0.0019162528,
 0.008565074,
 -0.00089677353,
 -0.0074268864,
 0.0042610937,
 -0.004550686,
 0.0018652364,
 -0.0039120135,
 -0.00040125148,
 0.0046343682,
 -0.005729739,
 -0.003462383,
 -0.0028425376,
 -0.011625722,
 6.525265e-05,
 0.0048296484,
 0.0011408408,
 -0.0037933814,
 -0.0030667975,
 0.002132826,
 0.0023482768,
 0.0042171567,
 -0.015214722,
 -0.0015119407,
 -0.009013695,
 -0.0042201863,
 -0.0018959688,
 -0.0003603017,
 0.01100151,
 0.0050300215,
 0.0048920433,
 -0.00094709755,
 -0.01289228,
 0.006284505,
 0.0058460752,
 -0.019651592,
 0.011964707,
 0.0032607005,
 -0.0013873084,
 0.00573607,
 0.00032672286,
 0.0015329956,
 0.0017873462,
 -0.010504306,
 0.004634965,
 -0.0031522787,
 -0.005527283,
 -0.0036865198,
 0.006878937,
 -0.0030253439,
 0.004641496,
 0.0075282445,
 0.004240525,
 -0.0058104196,
 7.3801726e-05,
 0.0029256819,
 0.0019397436,
 0.005238637,
 0.0047770897,
 -4.8900023e-05,
 0.0032055716,
 0.0012251831

In [35]:
def run_all_sig_tests(order='second'):
    exps = open_pickle(EXPERIMENT_DEFINITION_PATH)
    print(f'ORDER = {order}')
    for exp_num, exp in exps.items():
        print(f'Experiment: {exp_num}')
        X_terms = exp['X_terms']
        Y_terms = exp['Y_terms']
        A_terms = exp['A_terms']
        B_terms = exp['B_terms']
        if order == 'second':
            run_exps_2ndorder(X_terms, Y_terms, A_terms, B_terms, exp_num)
        else:
            run_exps_1storder(X_terms, Y_terms, A_terms, B_terms, exp_num)
run_all_exps(order='second')

TypeError: unhashable type: 'list'